In [1]:
import os
import sys
import time
import datetime
import sqlite3

import tweepy
import pandas as pd
import numpy as np
import yaml

In [2]:
root_dir = "../"
twitter_cred = yaml.load(open(root_dir + 'credentials/twitter.cred'))

auth = tweepy.OAuthHandler(twitter_cred['consumer_token'], 
                           twitter_cred['consumer_secret'])
auth.set_access_token(twitter_cred['access_token_key'], 
                      twitter_cred['access_token_secret'])

api = tweepy.API(auth)

In [4]:
con = sqlite3.connect("../data/trumps_followers.db")

In [5]:
cursor = con.execute("SELECT * FROM trumpsFollowers")

In [5]:
con.execute("""CREATE TABLE IF NOT EXISTS trumpsFollowers(id INTEGER PRIMARY KEY, screen_name, 
                                                          created_at, name, lang, friends_count, 
                                                          followers_count, statuses_count, favorites_count, 
                                                          profile_image_url, verified)""")

In [ ]:
def get_list(follower):
    created_at = datetime.datetime.strptime(follower._json['created_at'], 
                                            '%a %b %d %H:%M:%S %z %Y')
    created_at_str = created_at.strftime('%Y-%m-%d %H:%M:%S')
    return (follower._json['id'], follower._json['screen_name'], 
            created_at_str, follower._json['name'],
            follower._json['lang'], follower._json['friends_count'], 
            follower._json['followers_count'], follower._json['statuses_count'], 
            follower._json['favourites_count'], 
            follower._json['profile_image_url'], follower._json['verified'])

# https://dev.twitter.com/rest/reference/get/followers/list
# At this time, results are ordered with the most recent following first — however, 
# this ordering is subject to unannounced change and eventual consistency issues.
users = tweepy.Cursor(api.followers, screen_name="realDonaldTrump").items()

users_list = []
while True:
    try:
        follower = next(users)
        users_list.append(get_list(follower))
    except (tweepy.TweepError):
        print("Hit rate limit after %i users." % len(users_list))
        con.executemany("""INSERT OR REPLACE INTO trumpsFollowers(id, screen_name, 
                                                                  created_at, name, lang, 
                                                                  friends_count, followers_count, 
                                                                  statuses_count, favorites_count, 
                                                                  profile_image_url, verified) 
                            VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""", users_list)
        # Save changes to the database.
        con.commit()
        print("Writing new users and then waiting 15 minutes before getting more users.")
        time.sleep(900)
        users_list = []
        follower = next(users)
        users_list.append(get_list(follower))
    except StopIteration:
        con.close()
        break

Hit rate limit after 0 users.
Writing new users and then waiting 15 minutes before getting more users.
Hit rate limit after 300 users.
Writing new users and then waiting 15 minutes before getting more users.


In [39]:
cursor.execute('COMMIT')

OperationalError: cannot commit - no transaction is active

In [38]:
cursor.fetchall()

[(498014799,
  'paulamash',
  '2012-02-20 16:09:18',
  'paula mash',
  'en',
  104,
  3,
  5,
  1,
  'http://pbs.twimg.com/profile_images/631741808591159296/HolVkdCN_normal.jpg',
  0),
 (774479314549170176,
  'AlanSam94907874',
  '2016-09-10 05:27:12',
  'Alan Samuel',
  'es',
  88,
  0,
  0,
  0,
  'http://abs.twimg.com/sticky/default_profile_images/default_profile_2_normal.png',
  0)]